# Creation of artificial (synthetic) patient data

Note: This artificial data is intended only for use in exploring the methods, using up to 10 features. The method of synthethis does not maintain any covariance between features (as feature values are created independetly of each other, to eliminate any risk of identifying original data), though average feature values for patients at each hopsital are approximately maintained. These data may be used to train models with minimal loss of accuracy.

The key methodology is:

* Remove thrombolysis label
* Group original data by hopsital
    * For each of 10 features take bootstrap samples of that feature
* Combine data across hospitals
* Remove any duplicate rows, or rows that are identical to original data
* Train an XGBoost model on original data to predict use of thrombolysis
* Use the XGBoost model to leabl the synthetic data

## Load packages

In [47]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

## Load data

In [2]:
# Load data
data_loc = '../data/samuel_1/10k_training_test/'
original_data = pd.read_csv(data_loc + 'cohort_10000_train.csv')

# Get stroke teams = 
stroke_teams = list(set(original_data['StrokeTeam']))
stroke_teams.sort()


## Create unlabelled synthetic data by bootstrap sampling from individual feature values

In [3]:
cases = 1000
synthetic_data_list = []

# Sample data for each stroke team
for stroke_team in stroke_teams:
    # Set up data frame for synthetic team data
    synthetic_data = pd.DataFrame()
    
    # Get original team data
    mask = original_data['StrokeTeam'] == stroke_team
    team_data = original_data[mask]
    team_data_length = len(team_data)
    
    # Set team
    synthetic_data['StrokeTeam'] = np.repeat(stroke_team, cases)
    # Sample individual items from orioginal data with replacement

    # Arrival to image time
    synthetic_data['S2BrainImagingTime_min'] = np.random.choice(
        team_data['S2BrainImagingTime_min'], replace=True, size=cases)
    
    # Sample features independently from observed values

    synthetic_data['S2StrokeType_Infarction'] = np.random.choice(
        team_data['S2StrokeType_Infarction'], replace=True, size=cases)

    synthetic_data['S2NihssArrival'] = np.random.choice(
        team_data['S2NihssArrival'], replace=True, size=cases)
    
    synthetic_data['S2RankinBeforeStroke'] = np.random.choice(
        team_data['S2RankinBeforeStroke'], replace=True, size=cases)
    
    synthetic_data['AFAnticoagulent_Yes'] = np.random.choice(
        team_data['AFAnticoagulent_Yes'], replace=True, size=cases)

    synthetic_data['S1OnsetToArrival_min'] = np.random.choice(
        team_data['S1OnsetToArrival_min'], replace=True, size=cases)
    
    synthetic_data['S1AgeOnArrival'] = np.random.choice(
        team_data['S1AgeOnArrival'], replace=True, size=cases)
    
    # Use the same random index for S1OnsetTimeType_Precise and S1OnsetDateType_Stroke
    random_index = np.random.randint(0, team_data_length, size=cases)

    synthetic_data['S1OnsetTimeType_Precise'] = \
            [team_data['S1OnsetTimeType_Precise'].iloc[i] for i in random_index]
    
    synthetic_data['S1OnsetDateType_Stroke during sleep'] = \
            [team_data['S1OnsetDateType_Stroke during sleep'].iloc[i] for i in random_index]    

    # Add team data to full synthetic data
    synthetic_data_list.append(synthetic_data)

# Concatenate lists
synthetic_data_df = pd.concat(synthetic_data_list)

# Shuffle data
synthetic_data_df = synthetic_data_df.sample(frac=1)

In [4]:
image_time = 106
image_time = int((image_time + 5)/ 10) * 10
image_time

110

## Train a model on original data, to use to label synthetic data

In [5]:
# Load data
train = pd.read_csv(data_loc + 'cohort_10000_train.csv')
test = pd.read_csv(data_loc + 'cohort_10000_test.csv')

# Read in the names of the selected features for the model
number_of_features_to_use = 10
key_features = pd.read_csv('../data/samuel_1/feature_selection.csv')
key_features = list(key_features['feature'])[:number_of_features_to_use]
# And add the target feature name: S2Thrombolysis
key_features.append("S2Thrombolysis")

# Select features
train = train[key_features]
test = test[key_features]

# Get X and y
X_train = train.drop('S2Thrombolysis', axis=1)
X_test = test.drop('S2Thrombolysis', axis=1)
y_train = train['S2Thrombolysis']
y_test = test['S2Thrombolysis']

# One hot encode hospitals
X_train_hosp = pd.get_dummies(X_train['StrokeTeam'], prefix = 'team')
X_train = pd.concat([X_train, X_train_hosp], axis=1)
X_train.drop('StrokeTeam', axis=1, inplace=True)
X_test_hosp = pd.get_dummies(X_test['StrokeTeam'], prefix = 'team')
X_test = pd.concat([X_test, X_test_hosp], axis=1)
X_test.drop('StrokeTeam', axis=1, inplace=True)    

# Define model
model = XGBClassifier(verbosity=0, seed=42, learning_rate=0.5)

# Fit model
model.fit(X_train, y_train)

# Get predicted probabilities and class
y_probs = model.predict_proba(X_test)[:,1]
y_pred = y_probs > 0.5

# Show accuracy
accuracy = np.mean(y_pred == y_test)
print (f'Accuracy: {accuracy:.3f}')

Accuracy: 0.848


## Predict label for synthetic data set

In [6]:
# One hot encode hopsitals
X_features = key_features.copy(); X_features.remove('S2Thrombolysis')
X_synthetic = synthetic_data_df[X_features]
X_synthetic_hosp = pd.get_dummies(X_synthetic['StrokeTeam'], prefix = 'team')
X_synthetic = pd.concat([X_synthetic, X_synthetic_hosp], axis=1)
X_synthetic.drop('StrokeTeam', axis=1, inplace=True)

In [7]:
# Get predicted probabilities and class
y_probs = model.predict_proba(X_synthetic)[:,1]
y_pred = np.array([np.random.binomial(1, p) for p in y_probs])

# Ensure non-iscaemica stroke have no thrombolysis
mask = synthetic_data_df['S2StrokeType_Infarction'] == 0
y_pred[mask] = 0

synthetic_data_df['S2Thrombolysis'] = y_pred
# Save
synthetic_data_df.to_csv('./output/synthetic_10K_train.csv', index=False)

In [8]:
synthetic_data_df

,StrokeTeam,S2BrainImagingTime_min,S2StrokeType_Infarction,S2NihssArrival,S2RankinBeforeStroke,AFAnticoagulent_Yes,S1OnsetToArrival_min,S1AgeOnArrival,S1OnsetTimeType_Precise,S1OnsetDateType_Stroke during sleep,S2Thrombolysis
403,UIWEN7236N,58.0,1,2.0,0,1,96.0,72.5,1,0,0
611,LFZMO6561M,15.0,1,2.0,3,0,134.0,77.5,1,0,0
543,SBYTE4026H,93.0,1,28.0,1,0,79.0,47.5,1,0,0
569,QNARI2373R,32.0,1,4.0,3,0,29.0,87.5,1,0,1
971,GLRVJ5773V,28.0,1,4.0,0,0,79.0,87.5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
251,PSNKC7508G,24.0,1,5.0,1,1,60.0,72.5,1,0,0
643,IWINL6248B,1619.0,1,2.0,5,0,123.0,52.5,0,0,0
40,QNARI2373R,40.0,1,11.0,1,0,73.0,97.5,1,0,0
188,TBZOA3799I,10.0,1,2.0,5,0,210.0,82.5,1,0,0


## Test synthetic data to train model

In [9]:
# Load data
train = synthetic_data_df
test = pd.read_csv(data_loc + 'cohort_10000_test.csv')

# Select features
train = train[key_features]
test = test[key_features]

# Get X and y
X_train = train.drop('S2Thrombolysis', axis=1)
X_test = test.drop('S2Thrombolysis', axis=1)
y_train = train['S2Thrombolysis']
y_test = test['S2Thrombolysis']

# One hot encode hospitals
X_train_hosp = pd.get_dummies(X_train['StrokeTeam'], prefix = 'team')
X_train = pd.concat([X_train, X_train_hosp], axis=1)
X_train.drop('StrokeTeam', axis=1, inplace=True)
X_test_hosp = pd.get_dummies(X_test['StrokeTeam'], prefix = 'team')
X_test = pd.concat([X_test, X_test_hosp], axis=1)
X_test.drop('StrokeTeam', axis=1, inplace=True)    

# Define model
model = XGBClassifier(verbosity=0, seed=42, learning_rate=0.5)

# Fit model
model.fit(X_train, y_train)

# Get predicted probabilities and class
y_probs = model.predict_proba(X_test)[:,1]
y_pred = y_probs > 0.5

# Show accuracy
accuracy = np.mean(y_pred == y_test)
print (f'Accuracy: {accuracy:.3f}')

Accuracy: 0.844


In [10]:
# Add ROC here

## Examine distances between synthetic-real and real-real near neighbours

Distance are Cartesian distance between standardised data points.

### Function to standardise data

In [11]:
def standardise_data(X_train, X_test):
    
    # Initialise a new scaling object for normalising input data
    sc = StandardScaler() 

    # Set up the scaler just on the training set
    sc.fit(X_train)

    # Apply the scaler to the training and test sets
    train_std=sc.transform(X_train)
    test_std=sc.transform(X_test)
    
    return train_std, test_std

In [43]:
# Set up lists to store results
all_dist_real_real = []
all_dist_real_synthetic = []

for stroke_team in stroke_teams:

    # Get original team data
    mask = original_data['StrokeTeam'] == stroke_team
    team_original = original_data[mask][key_features]
    team_original.drop(['StrokeTeam', 'S2Thrombolysis'], axis=1, inplace=True)
    original_size = len(team_original)

    # Get synthetic data (match sample size to orginal data)
    mask = synthetic_data_df['StrokeTeam'] == stroke_team
    team_synthetic = synthetic_data_df[mask][key_features]
    team_synthetic.drop(['StrokeTeam', 'S2Thrombolysis'], axis=1, inplace=True)
    synthetic_size = len(team_synthetic)

    # Reduce larger set to size of small set
    if synthetic_size > original_size:
        team_synthetic = team_synthetic.sample(original_size)
    else:
        team_original = team_original.sample(synthetic_size)

    # Standardise
    team_original_std, team_synthetic_std = standardise_data(
        team_original, team_synthetic)

    # Get nearest neighbour distances in real data
    nn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(team_original_std)
    dists, idxs = nn.kneighbors()
    dist_real_real = dists.flatten()
    all_dist_real_real.append(dist_real_real)

    # Get closest real data point to synthetic data points
    dists, idxs = nn.kneighbors(team_synthetic_std)
    dist_real_synthetic = dists.flatten()
    all_dist_real_synthetic.append(dist_real_synthetic)

# Concatenate all distances
all_dist_real_real = np.concatenate(np.array(all_dist_real_real))
all_dist_real_synthetic = np.concatenate(np.array(all_dist_real_synthetic))